# IF702 Redes Neurais
Esse notebook contém um script base para o projeto da disciplina IF702 Redes Neurais.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Leitura e Limpeza dos Dados

A leitura do data set é feita utilizando a biblioteca `pandas`. O presente exemplo importa a base de dados `mammography`, assim, caso você esteja trabalhando com outro data set, modifique esta linha.
Para importar o conjunto de dados do PAKDD, use a função `pd.read_table` ao invés da `pd.read_csv`.

In [ ]:
data_set = pd.read_csv('data/mammography.csv.zip')
data_set.drop_duplicates(inplace=True)  # Remove exemplos repetidos

Separando o data set em atributos dependentes (X = features) e independentes (y = classe). No caso do `mammography` a classe majoritária está codificada como -1 e a classe minoritária está codificada como 1. Para treinar nossa rede neural precisamos que os valores de classe sejam 0 e 1, assim modificamos a codificação da majoritária para 0.

Perceba que esse pré-processamento varia de data set para data set.

In [ ]:
X = data_set.iloc[:, :-1].values
y = data_set.iloc[:, -1].values
y = np.where(y == -1, 0, 1)

Aqui dividimos o data set em treino, validação e teste.

In [ ]:
## Treino: 50%, Validação: 25%, Teste: 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, 
                                                    random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/3, 
                                                  random_state=42, stratify=y_train)

Para testar o comportamento da rede com diferentes funções de sampling, as mesmas devem ser implementadas e aplicadas ao conjunto de treinamento antes da normalização dos dados (você também pode investigar qual o efeito de aplicar o sampling após a normalização).

In [ ]:
## TO DO -- Implementar as funções de sampling a serem utilizadas

É importante lembrar de normalizar os dados. A classe `StandardScaler` centraliza as variáveis e transforma as features para terem variância unitária. Você pode testar outras opções como o `MinMaxScaler`.

Todas as alternativas estão disponíveis em:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Aqui definimos a arquitetura de nossa rede neural e treinamos ela.

A arquitetura da rede é definida como tendo apenas uma camada escondida. O código é bem intuitivo e a adição de novas camadas pode ser feita através da função `add`.

Várias funções de otimização estão disponíveis e seus parâmetros também podem ser definidos. 

Confira os exemplos em: https://keras.io/optimizers/

Um maior controle sobre quando a rede deve parar de treinar também pode ser exercido. 

Confira a documentação da classe `EarlyStopping`: https://keras.io/callbacks/

In [ ]:
# Cria o esboço da rede.
classifier = Sequential()
# Adiciona a primeira camada escondida contendo 16 neurônios e função de ativação tangente 
# hiperbólica. Por ser a primeira camada adicionada à rede, precisamos especificar a 
# dimensão de entrada (número de features do data set), no caso do mammography são 6.
classifier.add(Dense(16, activation='tanh', input_dim=6))
# Adiciona a camada de saída. Como nosso problema é binário, só precisamos de 1 neurônio 
# e função de ativação sigmoidal. A partir da segunda camada adicionada, keras já consegue 
# inferir o número de neurônios de entrada (nesse caso 16) e nós não precisamos mais 
# especificar.
classifier.add(Dense(1, activation='sigmoid'))
# Compila o modelo especificando o otimizador, a função de custo, e opcionalmente métricas 
# para serem observadas durante o treinamento.
classifier.compile(optimizer='adam', loss='mean_squared_error')
# Treina a rede, especificando o tamanho do batch, o número máximo de épocas, se deseja 
# parar prematuramente caso o erro de validação não decresça, e o conjunto de validação.
history = classifier.fit(X_train, y_train, batch_size=64, epochs=100000, 
                         callbacks=[EarlyStopping()], validation_data=(X_val, y_val))

Definição de funções auxiliares.

In [ ]:
def extract_final_losses(history):
    """Função para extrair o loss final de treino e validação.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    Dicionário contendo o loss final de treino e de validação.
    """
    return {'train_loss': history.history['loss'][-1], 'val_loss': history.history['val_loss'][-1]}

Usar a nossa rede para fazer predições e computar métricas de desempenho.

Mais métricas de desempenho: http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics

In [ ]:
## Fazer predições no conjunto de teste
y_pred = classifier.predict(X_test)
y_pred_class = classifier.predict_classes(X_test, verbose=0)

## Matriz de confusão
print('Matriz de confusão')
print(confusion_matrix(y_test, y_pred_class))

## Computar métricas de desempenho
losses = extract_final_losses(history)
print("\n{metric:<18}{value:.4f}".format(metric="Train Loss:", value=losses['train_loss']))
print("{metric:<18}{value:.4f}".format(metric="Validation Loss:", value=losses['val_loss']))
print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy_score(y_test, y_pred_class)))
print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall_score(y_test, y_pred_class)))
print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision_score(y_test, y_pred_class)))
print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1_score(y_test, y_pred_class)))
print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=roc_auc_score(y_test, y_pred)))